PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, ChatGPT)

In [ ]:
%pip install -q openai
%pip install -q langchain
%pip install -q -U langchain-openai

In [ ]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

In [ ]:
%pip install -q pypdf
%pip install -q chromadb
%pip install -q tiktoken

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
# PDF 로드
loader = PyPDFLoader("data/how-to-use-AI-in-6industries.pdf")
documents = loader.load()

In [ ]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
# OpenAI Embeddings 적용
embeddings = OpenAIEmbeddings()

# Chroma 벡터 저장소 생성
vector_store = Chroma.from_documents(texts, embeddings)
# 벡터 저장소에서 검색하기
# search_type="similarity" 옵션은 retrieval 객체에서 유사성 검색을 사용하여 질문 vector와 가장 유사한 문장 vector를 선택함
# search_kwargs 옵션은 vector 저장소에서 Prompt 2개의 텍스트 덩어리로 보내려는 것을 의미함
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# ChatOpenAI는 기본모델인  gpt-3.5-turbo 사용하고
# temperature=0은 보수적인 지문에 대한 답변을 내고, temperature=1 다양한 답변을 낼 수 있음 
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# RetrievalQA가 실질적인 RAG를 수행하는 객체
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # 기존 retriever 유지
    return_source_documents=True
)

In [ ]:
query = "언더라이팅"
query_result = chain.invoke(query)
print(type(query_result))
print(query_result)

In [ ]:
query_result['result']

In [ ]:
for key in query_result.keys():
    print(key)

In [ ]:
print(type(query_result['source_documents']))
type(query_result['source_documents'][0])

In [ ]:
doc = query_result['source_documents'][0]
print(type(doc))
doc_dict = doc.model_dump()
print(type(doc_dict))
print(doc_dict)

In [ ]:
for k in doc_dict:
    print(k)

In [ ]:
doc_dict['metadata']['title']

In [ ]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke(query)
print(query_result)

In [ ]:
print(query_result['result'])

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

system_template = """Use the following pieces of context to answer the user's question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{question}")
])


In [ ]:

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "summaries",
}

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # GPT-4를 사용하려면 model="gpt-4"

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    input_key="question"
)


In [ ]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke({"question": query})
print(type(query_result))
print(query_result)

In [ ]:
print(query_result['result'])

In [ ]:
query_result['source_documents']

In [ ]:
for doc in query_result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

In [ ]:
bot_message = query_result['result']
for i, doc in enumerate(query_result['source_documents']):
    bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '
    print(bot_message)

In [ ]:
%pip install -q gradio

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
import gradio as gr

# 1. PDF를 한 번만 로드하여 벡터 저장소 생성
def initialize_retriever():
    # PDF 파일을 로드하여 문서 객체로 변환
    loader = PyPDFLoader("how-to-use-AI-in-6industries.pdf")
    documents = loader.load()

    # 문서를 일정한 크기로 분할 (chunk_size=1000, 중첩 없음)
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # 문서의 텍스트를 벡터 임베딩으로 변환
    embeddings = OpenAIEmbeddings()
    # 변환된 벡터를 Chroma 벡터 저장소에 저장
    vector_store = Chroma.from_documents(texts, embeddings)
    # 저장된 벡터를 검색할 수 있는 retriever 생성 (유사한 문서 2개 검색)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})
    return retriever

# 2. 전역 retriever 생성 (앱 시작 시 한 번만 실행)
retriever = initialize_retriever()

# 3. 채팅 응답 함수 (retriever를 재사용)
def chat_respond(message, chat_history):
    #  시스템 메시지 템플릿: 문서 요약을 기반으로 질문에 답변하도록 설정
    system_template = """Use the following pieces of context to answer the user's question shortly.
    Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
    use "SOURCES" in capital letters regardless of the number of sources.
    If you don't know the answer, just say that "I don't know", don't try to make up an answer.
    ----------------
    {summaries}
    
    You MUST answer in Korean and in Markdown format:
    """
    # 사용자 질문을 받아 최종 Prompt 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])

    chain_type_kwargs = {
        "prompt": prompt, #  LLM이 사용할 프롬프트 지정
        "document_variable_name": "summaries", #  문서 요약 데이터를 LLM에 전달할 변수 이름
    }
    
    # OpenAI 모델 (GPT-3.5-turbo) 설정
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # RetrievalQA 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  #  문서를 하나의 큰 텍스트로 처리하는 방식
        retriever=retriever, #  유사 문서를 검색하는 retriever 연결
        return_source_documents=True, #  답변에 참조한 문서 정보 포함
        chain_type_kwargs=chain_type_kwargs,
        input_key="question" #  사용자 질문을 "question" 키로 전달
    )
    
    # LLM을 호출하여 질문에 대한 응답 생성
    query_result = chain.invoke({"question": message})

    # 모델의 답변을 가져오기
    bot_message = query_result['result']

    # 참조한 문서 정보를 응답에 추가
    for i, doc in enumerate(query_result['source_documents']):
        bot_message += f' [{i+1}] {doc.metadata.get("source", "Unknown")} (Page {doc.metadata.get("page", "N/A")})'

    # Gradio 채팅 기록 형식에 맞춰 응답을 저장
    chat_history.append({"role": "user", "content": message})  # 사용자 메시지 추가
    chat_history.append({"role": "assistant", "content": bot_message})  # 봇 응답 추가
    
    # 입력 창 초기화 및 갱신된 채팅 기록 반환
    return "", chat_history

# 4. # Gradio UI 생성 및 실행
with gr.Blocks() as demo:  
    # 채팅 창 (채팅 메시지를 표시하는 Gradio 컴포넌트)
    chatbot = gr.Chatbot(label="채팅창", type="messages")
    # 사용자 입력 텍스트 박스 (메시지를 입력하는 필드)
    msg = gr.Textbox(label="입력")
    # 초기화 버튼 (채팅 기록을 초기화하는 버튼)
    clear = gr.Button("초기화")

    # 사용자가 입력 필드에 메시지를 입력하면 chat_respond()가 실행됨
    #  - 입력한 메시지는 msg에서 가져옴
    #  - 기존 채팅 기록(chatbot)과 함께 전달됨
    #  - 응답을 받은 후, msg를 초기화하고 chatbot에 대화 내용을 추가
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # 초기화 버튼 클릭 시, 채팅 기록을 비우도록 설정
    clear.click(lambda: [], None, chatbot, queue=False)

# Gradio 앱 실행 (debug 모드 활성화)
demo.launch(debug=True)  